In [6]:
# !pip install langchain, openai, pinecone-datasets, pinecone-client, trulens_eval
# !pip install python-dotenv -q

In [7]:
# from google.colab import drive
from dotenv import load_dotenv, find_dotenv
import os

# Монтируем Google Drive
# drive.mount('/content/drive')

# # Загружаем .env файл
# load_dotenv('/content/drive/My Drive/python/.env')
load_dotenv(find_dotenv())

# Пример использования переменной среды
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

### Creating the Index in Pinecone

In [8]:
# import pinecone_datasets

# dataset = pinecone_datasets.load_dataset('wikipedia-simple-text-embedding-ada-002-100K')
# dataset.head()

In [9]:
import os
from pinecone import Pinecone, ServerlessSpec

# Установите соединение с Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [10]:
import pinecone

index_name_v1 = "wiki-test-index"

# # Создайте индекс
# pc.create_index(
#     name=index_name_v1,
#     dimension=1536,
#     metric='cosine',
#     spec=ServerlessSpec(
#         cloud='aws',
#         region='us-west-2'
#     )
# )

In [11]:
# Получите экземпляр индекса
index = pc.Index(index_name_v1)

# for batch in dataset.iter_documents(batch_size=100):
#     index.upsert(batch)

### Building the Vector Store

In [12]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [13]:
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key=OPENAI_API_KEY
)

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/Users/valentinshapovalov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/valentinshapovalov/opt/anaconda3/lib/python3.9/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Initialize our RAG application

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

/Users/valentinshapovalov/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### TruLens for Evaluation and Tracking of LLM Experiments

In [24]:
!pip uninstall -y trulens_eval
!pip install trulens_eval

Found existing installation: trulens-eval 0.25.0
Uninstalling trulens-eval-0.25.0:
  Successfully uninstalled trulens-eval-0.25.0
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.29-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.28-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.27-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.26-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.25-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.24-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_core-0.1.23-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.0.87-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 878.4 kB/s eta 0:00:00a 

In [26]:
# Imports main tools for eval
from trulens_eval import TruChain, Feedback, Tru, feedback, Select
import numpy as np
tru = Tru()

# OpenAI as feedback provider
openai = feedback.OpenAI()

# Question/answer relevance between overall question and answer.
qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
qs_relevance = Feedback(openai.qs_relevance).on_input().on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content).aggregate(np.mean)


VersionConflict: Package pydantic is installed but has a version conflict:
	(pydantic 1.10.14 (/Users/valentinshapovalov/opt/anaconda3/lib/python3.9/site-packages), Requirement.parse('pydantic<3,>=2'))

This package is required for trulens_eval. Please resolve the conflict by
installing a compatible version with:

    ```bash
    pip install 'pydantic<3,>=2'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```


In [ ]:
# wrap with TruLens
truchain = TruChain(qa,
    app_id='Chain1_WikipediaQA',
    feedbacks=[qa_relevance, qs_relevance])

truchain(“Which state is Washington D.C. in?”)

In [ ]:
tru.run_dashboard()